# TEASER++ with headless

Jupyter may not support rendering in some case but we need output result, so we need to set make options in `CMake`. Before start, you should clone `TEASER-plusplus` repository first :

In [ ]:
!git clone --recursive https://github.com/MIT-SPARK/TEASER-plusplus.git

Create build directory :

In [1]:
!cd TEASER-plusplus && mkdir -p build && cd build

Set build options:

In [1]:
!cd TEASER-plusplus/build && cmake -DENABLE_HEADLESS_RENDERING=ON \
                 -DBUILD_GUI=OFF \
                 -DUSE_SYSTEM_GLEW=OFF \
                 -DUSE_SYSTEM_GLFW=OFF \
                 .. && make teaserpp_python

# [optional for normal case] :
# !cd TEASER-plusplus/build && cmake -DTEASERPP_PYTHON_VERSION=3.6 .. && make teaserpp_python

-- Enable printing of diagnostic messages.
-- Configuring done
-- Generating done
-- Build files have been written to: /app/notebook/bom/TEASER-plusplus/build/googletest-download
[100%] Built target googletest
CMake Deprecation Warning at build/googletest-src/CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


CMake Deprecation Warning at build/googletest-src/googlemock/CMakeLists.txt:42 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


CMake Deprecation Warning at build/googletest-src/googletest/CMakeLists.txt:49 (cmake_minimum_required):
  Compatibi

Install dependencies:

In [2]:
!cd TEASER-plusplus/build && cd python && pip install .

Processing /app/notebook/bom/TEASER-plusplus/build/python
  Created wheel for teaserpp-python: filename=teaserpp_python-1.0.0-py3-none-any.whl size=96787 sha256=f9e2ea4de487bf147bf4b0c8d0ba4ab20e05427163c7d25994e47ae914724582
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ov5nct4/wheels/63/7f/01/46fd10fab12e4155da32528e9bc54173b032a2639ced68522e
Successfully built teaserpp-python
  Attempting uninstall: teaserpp-python
    Found existing installation: teaserpp-python 1.0.0
    Uninstalling teaserpp-python-1.0.0:
      Successfully uninstalled teaserpp-python-1.0.0


In [3]:
!pip3 -q install open3d==0.9.0 scikit-learn numpy matplotlib opencv-python

Let's test robust registration :

In [4]:
!cd TEASER-plusplus/examples/teaser_python_ply && \
python teaser_python_ply.py

# !cd TEASER-plusplus/examples/teaser_python_3dsmooth && \
# python teaser_python_3dsmooth.py

        TEASER++ Python registration example      
Starting scale solver.
Scale estimation complete.
Max core number: 751
Num vertices: 1890
*** [pmc heuristic: thread 1]   current max clique = 602,  time = 0.00973892 sec
*** [pmc heuristic: thread 1]   current max clique = 607,  time = 0.086772 sec
*** [pmc heuristic: thread 10]   current max clique = 609,  time = 0.109637 sec
*** [pmc heuristic: thread 27]   current max clique = 610,  time = 0.178972 sec
[pmc heuristic]	 mc = 610
Created adjacency matrix in 0.00241804 seconds
[pmc: initial k-core pruning]  before pruning: |V| = 1889, |E| = 327867
[pmc: initial k-core pruning]  after pruning:  |V| = 1889, |E| = 327818
[pmc]  initial pruning took 0.00223303 sec
[pmc: sorting neighbors]  |E| = 655636, |E_sorted| = 655636
|V| = 814
>>> [pmc: thread 20]
[pmc: bounds updated - thread [pmc: bounds updated - thread 31]  time = 130.019861 sec, ]  time = |V| = 1879 (10 / 1889), |E| = 327818, w = 610, p = 0.183835, d_min = 0, d_avg = 0.019877 s

---------------------------------------

### Memory issue

Technically, It should not consider as a bug, more like an algorithm failure.
I compare Teaser++ with ICP kind registration like the T-RO.
ICP kind method can handle 50k points (nearest point correspondence ) with no problem.
However, Teaser++ was crushed with a huge memory leak.
I trace it to

registration.cc

```c
   computeTIMs(const Eigen::Matrix<double, 3, Eigen::Dynamic>& v,
                                          Eigen::Matrix<int, 2, Eigen::Dynamic>* map)
```

and find
map->resize(2, N * (N - 1) / 2);
Eigen::Matrix<double, 3, Eigen::Dynamic> vtilde(3, N * (N - 1) / 2);

and in loop

```c
// populate the index map
Eigen::Matrix<int, 2, Eigen::Dynamic> map_addition(2, N);
for (size_t j = 0; j < N; ++j) {
  map_addition(0, j) = i;
  map_addition(1, j) = j;
}
```

if N = 50k, vtilde,map will be 500000^2 / 2.
In the following loop, the memory reached its limit.

Ref. https://github.com/MIT-SPARK/TEASER-plusplus/issues/70

### [Optional] Test with our custom script

I provided custom script(in current directory) to test registration function, you can pass 3 arguments:
 - source point cloud file.
 - target point cloud file.
 - number of point of point cloud
 
note: the script just an example, don't use in production.

In [9]:
!python teaser_pp.py \
'060c4f8c4dec4c7688d713bde9f0cfab_0.ply' \
'1611745410data_060c4f8c4dec4c7688d713bde9f0cfab_frame0.ply' \
2000

        TEASER++ Python registration example      
src_pcs: 060c4f8c4dec4c7688d713bde9f0cfab_0.ply
dst_pcs: 1611745410data_060c4f8c4dec4c7688d713bde9f0cfab_frame0.ply
src: (3, 2000)
dst: (3, 2000)
Starting scale solver.
Scale estimation complete.
Max core number: 2
Num vertices: 2001
*** [pmc heuristic: thread 4]   current max clique = 3,  time = 5.79357e-05 sec
[pmc heuristic]	 mc = 3
Max Clique of scale estimation inliers: 
Using chain graph for GNC rotation.
Starting rotation solver.
GNC rotation estimation noise bound:0.1
GNC rotation estimation noise bound squared:0.01
GNC-TLS terminated because maximum residual at initialization is very small.
Rotation estimation complete.
Starting translation solver.
Translation estimation complete.
          TEASER++ Results           
Estimated rotation: 
[[ 0.78706034  0.39811942  0.47120797]
 [-0.60617249  0.64081732  0.47107119]
 [-0.11441564 -0.65639476  0.74569094]]
Estimated translation: 
[-106.51897544  -74.71157668 -312.90982502]
Time 